In [1]:
import numpy as np
import os
import sys
import pyscf
from qiskit import QuantumCircuit
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

### import nature library
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper
import time
import inspect
from pprint import pprint

In [2]:
# Define the molecule
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Run the driver to get the problem
problem = driver.run()

# Define the log output (here using stdout)
flog = sys.stdout

### Apply freezecore transformer
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

hamiltonian = problem.hamiltonian.second_q_op()
print(hamiltonian)

Fermionic Operator
number spin orbitals=4, number terms=36
  0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.33229086512764805 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.33229086512764805 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211567 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211567 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211567 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211567 * ( +_0 +_3 -_2 -_1 )
+ 0.09046559989211567 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211567 * ( +_1 +_1 -_0 -_0 )
+ 0.09046559989211567 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211567 * ( +_1 +_3 -_2 -_0 )
+ 0.33229086512764805 * ( +_1 +_0 -_0 -_1 )
+ 0.3492868613660088 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764805 * ( +_1 +_2 -_2 -_1 )
+ 0.3492868613660088 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.33229086512764805 * ( +_2 +_1 -_1 -_2 )
+ 0.33785507740175813 * ( +_2 +_2 -_2 -_2 )
+ 0.33229086512764805 * ( +_2 +_3 -_3 -_2 )
+ 0.09046559989211567 * ( +_2 +_0 -_1 -_3 )
+ 0.090465599892115

In [10]:
print(problem.num_particles)
print(problem.num_spatial_orbitals)
#print(problem.active_orbitals)

(1, 1)
2


In [12]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2,2) # syntax = AST(num_electrons,num_orbitals)
as_problem = as_transformer.transform(problem)

print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(1, 1)
2
Polynomial Tensor
 "+-":
array([[-1.25633907e+00,  0.00000000e+00],
       [-8.32667268e-17, -4.71896007e-01]])
 "++--":
array([[[[6.75710155e-01, 2.22044605e-16],
         [2.22044605e-16, 6.64581730e-01]],

        [[2.22044605e-16, 1.80931200e-01],
         [1.80931200e-01, 1.11022302e-16]]],


       [[[2.22044605e-16, 1.80931200e-01],
         [1.80931200e-01, 1.11022302e-16]],

        [[6.64581730e-01, 1.11022302e-16],
         [1.11022302e-16, 6.98573723e-01]]]])


In [13]:
hamiltonian = as_problem.hamiltonian.second_q_op()

In [14]:
#convert second quantized operator into pauli strings using any chemistry encoding of choice. we use jw.
mapper = JordanWignerMapper()

pauli_hamiltonian = mapper.map(hamiltonian)

for pauli, coeff in sorted(pauli_hamiltonian.label_iter()):
    print(f"{coeff.real:+.8f} * {pauli}")

-0.81054798 * IIII
+0.17218393 * IIIZ
-0.22575349 * IIZI
+0.12091263 * IIZZ
+0.17218393 * IZII
+0.16892754 * IZIZ
+0.16614543 * IZZI
+0.04523280 * XXXX
+0.04523280 * XXYY
+0.04523280 * YYXX
+0.04523280 * YYYY
-0.22575349 * ZIII
+0.16614543 * ZIIZ
+0.17464343 * ZIZI
+0.12091263 * ZZII


In [15]:
pauli_hamiltonian

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.12091263+0.j,
  0.17218393+0.j,  0.16892754+0.j, -0.22575349+0.j,  0.16614543+0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])

In [16]:
eig = np.linalg.eigvals(pauli_hamiltonian)
print('NUMPY eig',np.min(eig))

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=pauli_hamiltonian)
print('VQE eig', result.eigenvalue)

NUMPY eig (-1.8572750302023788+0j)
VQE eig -1.8572602858554217


In [17]:
n_qpe_qbits = 10

U = HamiltonianGate(pauli_hamiltonian, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
#measure_circ.draw("mpl")

In [18]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

{'1011001100': 2, '1011001000': 2, '1101001100': 2, '0111001100': 1, '0000000000': 3}


In [19]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

MAX count: ('0000000000', 3)
Theta value: 0.0
QPE-approximated U-eigenvalue: (1+0j)
QPE-approximated H-eigenvalue: -0.0
